In [1]:
import sys

import datetime
import pickle

import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity



import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [2]:

df_crime_tot =pd.read_csv('../Data/Mine/Crime_TOTAL_US_1960_2014_Added_per100000.csv')

df_crime_tot



,Year,Population,Violent crime total,Murder and nonnegligent Manslaughter,Murder_per100000,Legacy rape /1,Revised rape /2,Robbery,Aggravated assault
0,1960,179323175,288460,9110.0,5.080213,17190.0,NaN,107840.0,154320.0
1,1961,182992000,289390,8740.0,4.776165,17220.0,NaN,106670.0,156760.0
2,1962,185771000,301510,8530.0,4.591675,17550.0,NaN,110860.0,164570.0
3,1963,188483000,316970,8640.0,4.583968,17650.0,NaN,116470.0,174210.0
4,1964,191141000,364220,9360.0,4.896909,21420.0,NaN,130390.0,203050.0
5,1965,193526000,387390,9960.0,5.146595,23410.0,NaN,138690.0,215330.0
6,1966,195576000,430180,11040.0,5.644864,25820.0,NaN,157990.0,235330.0
7,1967,197457000,499930,12240.0,6.198818,27620.0,NaN,202910.0,257160.0
8,1968,199399000,595010,13800.0,6.920797,31670.0,NaN,262840.0,286700.0
9,1969,201385000,661870,14760.0,7.329245,37170.0,NaN,298850.0,311090.0


In [4]:

df_crime_state_by_state =pd.read_csv('../Data/Mine/CrimeStatebyState_1960-2014_added_per100000.csv')

df_crime_state_by_state

,State,Year,Population,Violent_crime_total,Murder_and_nonnegligent_Manslaughter,Murder_per100000,Legacy_rape_/1,Revised_rape_/2,Robbery,Aggravated_assault
0,Alabama,1960,3266740,6097,406,12.428292,281,NaN,898,4512
1,Alabama,1961,3302000,5564,427,12.931557,252,NaN,630,4255
2,Alabama,1962,3358000,5283,316,9.410363,218,NaN,754,3995
3,Alabama,1963,3347000,6115,340,10.158351,192,NaN,828,4755
4,Alabama,1964,3407000,7260,316,9.275022,397,NaN,992,5555
5,Alabama,1965,3462000,6916,395,11.409590,367,NaN,992,5162
6,Alabama,1966,3517000,8098,384,10.918396,341,NaN,1124,6249
7,Alabama,1967,3540000,8448,415,11.723164,371,NaN,1167,6495
8,Alabama,1968,3566000,8288,421,11.805945,396,NaN,1462,6009
9,Alabama,1969,3531000,8842,485,13.735486,494,NaN,1448,6415


In [5]:
df_sected_crime = df_crime_state_by_state[df_crime_state_by_state['State']== 'Alabama' ]

df_sected_crime

,State,Year,Population,Violent_crime_total,Murder_and_nonnegligent_Manslaughter,Murder_per100000,Legacy_rape_/1,Revised_rape_/2,Robbery,Aggravated_assault
0,Alabama,1960,3266740,6097,406,12.428292,281,NaN,898,4512
1,Alabama,1961,3302000,5564,427,12.931557,252,NaN,630,4255
2,Alabama,1962,3358000,5283,316,9.410363,218,NaN,754,3995
3,Alabama,1963,3347000,6115,340,10.158351,192,NaN,828,4755
4,Alabama,1964,3407000,7260,316,9.275022,397,NaN,992,5555
5,Alabama,1965,3462000,6916,395,11.409590,367,NaN,992,5162
6,Alabama,1966,3517000,8098,384,10.918396,341,NaN,1124,6249
7,Alabama,1967,3540000,8448,415,11.723164,371,NaN,1167,6495
8,Alabama,1968,3566000,8288,421,11.805945,396,NaN,1462,6009
9,Alabama,1969,3531000,8842,485,13.735486,494,NaN,1448,6415


In [6]:
# Create and style traces

data = []

trace0 = go.Scatter(
    x = df_crime_tot['Year'],
    y = df_crime_tot['Murder_per100000'],
    name = 'tot. US crime',
#     line = dict(
#         color = ('rgb(205, 12, 24)'),
#         width = 4)
)
data.append(trace0)

for state in df_crime_state_by_state.State.unique():
    
    df_sected_crime = df_crime_state_by_state[df_crime_state_by_state['State']== state ]
    
    
    trace1 = go.Scatter(
        x = df_sected_crime['Year'],
        y = df_sected_crime['Murder_per100000'],
        name = state,

    )
    data.append(trace1)

    
# Edit the layout
layout = dict(
#     title = 'Average High and Low Temperatures in New York',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Average number of murders per 100,000 people'),
              )

fig = dict(data=data, layout=layout)



offline.plot(fig, auto_open=True, image = 'png', image_filename="time_evol_tot_crime" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/time_evol_tot_crime.html', validate=True)




'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/time_evol_tot_crime.html'

In [44]:
df_sected_crime = df_crime_state_by_state[df_crime_state_by_state['State']!= 'District of Columbia' ]

df_sected_crime = df_sected_crime[df_sected_crime['Year']== 2000 ]
  
 
    
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]


data = [ dict(
        type='choropleth',
        colorscale = scl,
       # autocolorscale = False,
        locations = df_sected_crime['State'],
        z = df_sected_crime['Murder_per100000'],
        locationmode = 'USA-states',
        text = df_sected_crime['State'],
#         marker = dict(
#             line = dict (
#                 color = 'rgb(255,255,255)',
#                 width = 2
#             ) ),
        colorbar = dict(
            title = "murders per 100,000 people")
        ) ]

layout = dict(
        #title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )


offline.plot(fig, auto_open=True, image = 'png', image_filename="d3-cloropleth-map" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/d3-cloropleth-map.html', validate=True)




'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/d3-cloropleth-map.html'

In [36]:
df_sected_crime.Murder_per100000.mean()

4.6415230832420002

In [46]:

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['state'] + '<br>' +\
    'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
    'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
    'Wheat '+df['wheat']+' Corn '+df['corn']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['total exports'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

